In [6]:
#!/usr/bin/env python

import os
import re
import csv
import fnmatch
import numpy as np
import pandas as pd
import matplotlib as mplt
import matplotlib.pyplot as plt
from scipy import special as sp
from scipy.optimize import curve_fit

In [7]:
# Path of directory of this script is
# 'G:\\Il mio Drive\\Corsi\\Object_Oriented_Programming\\Python\\Process_claro_py\\'

# Ask for top directory
TDIR = 'C:\\Users\\MARTINA\\Desktop\\secondolotto_1\\'
# C:\Users\MARTINA\Desktop\secondolotto_1\
# TDIR = input("\nEnter the top directory's abs path (end with /):\n> ")

In [3]:
TDIR

'C:\\Users\\MARTINA\\Desktop\\secondolotto_1\\'

In [9]:
TDIR = input("Please provide top directory absolute path. "
             +"If you're a Windows user, provide path with backslashes \"\\\", double if whitespace is present in the path.\n")
# if isinstance(TDIR, str):
#     raise NameError("Plase provide a string.")
        
DIRPAT = "*Station_1__11\Station_1__??_Summary\Chip_001\S_curve"
        # "*Station_1__*\Station_1__??_Summary\Chip_*\S_curve" # this is the generic version
FILEPAT = "Ch_*_offset_*_Chip_*.txt"
OUTFILE = "claro_files.txt"
OUTBAD = 'bad_files.txt'
fileinfos = dict()

for root, dirs, files in os.walk(TDIR):
    if fnmatch.fnmatch(
        root, TDIR + DIRPAT):
        for f in files:
            if fnmatch.fnmatch(f, FILEPAT):
                thisfile = os.path.join(root,f)
                        # path of file currently used
                with open(thisfile) as csvfile:
                    lines = csvfile.readlines()
                    firstline = lines[0].split()
                    try:
                        if isinstance(float(firstline[2]), float):
                            temp = re.findall("[0-9]+", thisfile)
                            fileinfos[thisfile] = {
                                    'station': temp[0],
                                    'sub': temp[2],
                                    'chip': temp[5],
                                    'ch': temp[6],
                                    'offset': temp[7],
                                    'transition': firstline[1],
                                    'width': firstline[2],
                                    }
                        with open(OUTFILE, "w+") as output:
                            output.write(thisfile+"\n")
                    except ValueError:
                        print(f"ValueError: Couldn't read data in: {thisfile}\n "
                             + "Going on...")
                        with open(OUTBAD, "w+") as output:
                            output.write(thisfile+"\n")
                        pass
                    except IndexError:
                        raise

print(f"Completed. Output paths to good files are stored in {OUTFILE}")
print(f"Output paths to bad files are stored in {OUTBAD}")

Please provide top directory absolute path. If you're a Windows user, provide path with backslashes "\", double if whitespace is present in the path.
 C:\Users\MARTINA\Desktop\secondolotto_1\


ValueError: Couldn't read data in: C:\Users\MARTINA\Desktop\secondolotto_1\Station_1__11\Station_1__11_Summary\Chip_001\S_curve\Ch_8_offset_0_Chip_001.txt
 Going on...
Completed. Output paths to good files are stored in claro_files.txt
Output paths to bad files are stored in claro_files.txt


In [62]:
def claro_read(TDIR, DIRPATH="*Station_1__*\Station_1__??_Summary\Chip_*\S_curve",
               FILEPATH="Ch_*_offset_*_Chip_*.txt"):
    """
    [Description of function and input parameters]
    """
    #################### To limit the function on 1 folder only
    DIRPATH  = "*Station_1__11\Station_1__??_Summary\Chip_001\S_curve"
    ####################
    
    if isinstance(TDIR, str)==False:
        raise NameError('Please provide a string with delimiter: double backslash (\\\) as top directory path.')

    enter    = input(f'The default subfolders\' paths are {DIRPATH}.\n'
                    +f'The default file names are {FILEPATH}.\n'
                    +f'To confirm, press Enter. Press any other key to change the paths.\n')
    if enter!='':
        DIRPATH  = input('Please provide subfolders\' paths. You may use wildcards.\n')
        FILEPATH = input('Please provide file names. You may use wildcards.\n')
    
    OUTFILE = "claro_files.txt"
    OUTBAD  = 'bad_files.txt'
    fileinfos = dict()
    
    for root, dirs, files in os.walk(TDIR):
        if fnmatch.fnmatch(
            root, TDIR + DIRPAT):
            for f in files:
                if fnmatch.fnmatch(f, FILEPAT):
                    thisfile = os.path.join(root,f)
                            # path of file currently used
                    with open(thisfile) as csvfile:
                        lines = csvfile.readlines()
                        firstline = lines[0].split()
                        try:
                            if isinstance(float(firstline[2]), float):
                                temp = re.findall("[0-9]+", thisfile)
                                fileinfos[f'Ch.{temp[6]}'] = {
                                        'path' : thisfile,
                                        'station': temp[0],
                                        'sub': temp[2],
                                        'chip': temp[5],
                                        'ch': temp[6],
                                        'offset': temp[7],
                                        'amplitude': float(firstline[0]),
                                        'transition': float(firstline[1]),
                                        'width': float(firstline[2]),
                                        }
                            with open(OUTFILE, "w+") as output:
                                output.write(thisfile+"\n")
                        except ValueError:
                            print(f"ValueError: Couldn't read data in: {thisfile}\n "
                                 + "Going on...")
                            with open(OUTBAD, "w+") as output:
                                output.write(thisfile+"\n")
                            pass
                        except IndexError:
                            raise
    
    print(f"Completed. Output paths to good files are stored in {OUTFILE}")
    print(f"Output paths to bad files are stored in {OUTBAD}")
    
    return pd.DataFrame(fileinfos)

In [63]:
a = claro_read('C:\\Users\\MARTINA\\Desktop\\secondolotto_1\\')

The default subfolders' paths are *Station_1__11\Station_1__??_Summary\Chip_001\S_curve.
The default file names are Ch_*_offset_*_Chip_*.txt.
To confirm, press Enter. Press any other key to change the paths.
 


ValueError: Couldn't read data in: C:\Users\MARTINA\Desktop\secondolotto_1\Station_1__11\Station_1__11_Summary\Chip_001\S_curve\Ch_8_offset_0_Chip_001.txt
 Going on...
Completed. Output paths to good files are stored in claro_files.txt
Output paths to bad files are stored in bad_files.txt


In [64]:
a

,Ch.0,Ch.1,Ch.2,Ch.3,Ch.4,Ch.5,Ch.6,Ch.7
path,C:\Users\MARTINA\Desktop\secondolotto_1\Statio...,C:\Users\MARTINA\Desktop\secondolotto_1\Statio...,C:\Users\MARTINA\Desktop\secondolotto_1\Statio...,C:\Users\MARTINA\Desktop\secondolotto_1\Statio...,C:\Users\MARTINA\Desktop\secondolotto_1\Statio...,C:\Users\MARTINA\Desktop\secondolotto_1\Statio...,C:\Users\MARTINA\Desktop\secondolotto_1\Statio...,C:\Users\MARTINA\Desktop\secondolotto_1\Statio...
station,1,1,1,1,1,1,1,1
sub,11,11,11,11,11,11,11,11
chip,001,001,001,001,001,001,001,001
ch,0,1,2,3,4,5,6,7
offset,0,0,0,0,0,0,0,0
amplitude,1000.767211,1000.07304,1000.635659,1000.006139,1001.002002,1000.742599,1001.054753,1001.014499
transition,161.272238,233.74269,181.384648,180.409687,193.324689,195.855577,187.73549,186.527393
width,-1.923543,-1.841865,-1.965088,-1.734234,-1.848843,-2.029422,-2.015199,-2.256561


In [65]:
isinstance(a.iloc[6,0], float)

True